In [ ]:
# This file converts object detection annotations from YOLO format (.txt and .names) to COCO format (.json)
# The base code is originally from: https://www.programmersought.com/article/76707275021/
# It was modified extensively as the original code was not working for my needs.
# More information about the conversion process can be found here: https://prabhjotkaurgosal.com/weekly-learning-blogs/

In [1]:
import os
import json
import cv2
import random
import time

In [ ]:
# (OPTIONAL) step
# Preparing the Dataset
# Convert all images to .jpg (This is not necessary but made my life easier down the line)

from PIL import Image
#iml = Image.open(r'/absolute/path/to/the/image/directory/followed/by/image/name/with/its/extesnion')
#rgb_im = iml.convert('RGB')
#rgb_im.save(r'/absolutae/path/to/the/directory/where/image/must/be/stored/image_name.jpg')


In [ ]:
# (OPTIONAL) step
# Preparing the Dataset - remove unlabelled files
# I chose to remove the image files from the dataset that were not labelled
# Move the files (jpg and txt that are not labelled) from the working directory and put them in another folder. . .

# import shutil

# directory_labels = os.fsencode("/home/salem/datasets/iSAID/images/test/images") #absolute path to the directory where all labels are stored
# directory_images = os.fsencode("/home/salem/datasets/iSAID/images/test/labels") #absolute path to the directory where all images are stored
# directory_unlabelled_images = os.fsencode("/home/salem/datasets/iSAID/images/test/unlabelledImages") #absolute path to the directory where unlabelled images are to be stored

# not_labelled_files = 0
# for file in os.listdir(directory_labels): #Read from the directory where the labels are stored
#      filename = os.fsdecode(file)
#      if filename.endswith(".txt"):
#         yolo_annotation_path = (os.path.join(directory_labels.decode("utf-8"), filename))
#         base=os.path.basename(yolo_annotation_path)
#         file_name_without_ext = os.path.splitext(base)[0] # name of the file without the extension
#         img_path = os.path.join(directory_images.decode("utf-8"), file_name_without_ext+ "." + 'png')
        
#         filesize = os.path.getsize(yolo_annotation_path)
#         if filesize == 0: #chek if the label file (.txt) is empty - that is it has no labels
#             UnlabeledImg_path = os.path.join(directory_unlabelled_images.decode("utf-8"), file_name_without_ext+ "." + 'png')
#             not_labelled_files = not_labelled_files +1
            
#             shutil.move(img_path, UnlabeledImg_path) #remove the corresponding image to another directory
# print("The number of unlabelled files is: ", not_labelled_files)

In [ ]:
# #### This is where the conversion process starts from YOLO to COCO format

# # Category file, one category per line
# yolo_format_classes_path = '/home/salem/datasets/iSAID/images/train2/obj.names'
# # Write the category according to your own data set. 

# #Read the categories file and extract all categories
# with open(yolo_format_classes_path,'r') as f1:
#     lines1 = f1.readlines()
# categories = []
# for j,label in enumerate(lines1):
#     label = label.strip()
#     categories.append({'id':j+1,'name':label,'supercategory': label})
    
# write_json_context = dict()
# write_json_context['info'] = {'description': '', 'url': '', 'version': '', 'year': 2021, 'contributor': '', 'date_created': '2021-02-12 11:00:08.5'}
# write_json_context['licenses'] = [{'id': 1, 'name': None, 'url': None}]
# write_json_context['categories'] = categories
# write_json_context['images'] = []
# write_json_context['annotations'] = []

In [2]:
#### This is where the conversion process starts from YOLO to COCO format

# Category file, one category per line
yolo_format_classes_path = '/home/salem/datasets/iSAID/images/train/obj.names'
# Write the category according to your own data set. 

#Read the categories file and extract all categories
with open(yolo_format_classes_path,'r') as f1:
    lines1 = f1.readlines()
categories = []
categories_names = []
for j,label in enumerate(lines1):
    label = label.strip()
    categories.append({'id':j+1,'name':label})
    
write_json_context = dict()
# write_json_context['info'] = {'description': '', 'url': '', 'version': '', 'year': 2021, 'contributor': '', 'date_created': '2021-02-12 11:00:08.5'}
# write_json_context['licenses'] = [{'id': 1, 'name': None, 'url': None}]
write_json_context['images'] = []
write_json_context['categories'] = categories
write_json_context['annotations'] = []

In [ ]:
# # Read the YOLO formatted label files (.txt) to extarct bounding box information and store in COCO format

# #Read the label files (.txt) to extarct bounding box information and store in COCO format
# directory_labels = os.fsencode("/home/salem/datasets/iSAID/images/train2/labels")
# #directory_images = os.fsencode("/home/jyoti/Desktop/csc8800/datasets/DoorDetectDataset/test")
# directory_images = os.fsencode("/home/salem/datasets/iSAID/images/train2/images")

# file_number = 1
# num_bboxes = 1
# for file in os.listdir(directory_images):
#     filename = os.fsdecode(file)
#     if filename.endswith(".png"):
#         img_path = (os.path.join(directory_images.decode("utf-8"), filename))
#         base=os.path.basename(img_path)
#         file_name_without_ext = os.path.splitext(base)[0] # name of the file without the extension
#         yolo_annotation_path  = os.path.join(directory_labels.decode("utf-8"), file_name_without_ext+ "." + 'txt')
#         img_name = os.path.basename(img_path) # name of the file without the extension
#         img_context = {}
#         height,width = cv2.imread(img_path).shape[:2]
#         img_context['file_name'] = img_name
#         img_context['height'] = height
#         img_context['width'] = width
#         img_context['date_captured'] = '2021-02-12 11:00:08.5'
#         img_context['id'] = file_number # image id
#         img_context['license'] = 1
#         img_context['coco_url'] =''
#         img_context['flickr_url'] = ''
#         write_json_context['images'].append(img_context)
        
#         with open(yolo_annotation_path,'r') as f2:
#             lines2 = f2.readlines() 

#         for i,line in enumerate(lines2): # for loop runs for number of annotations labelled in an image
#             line = line.split(' ')
#             bbox_dict = {}
#             class_id, x_yolo,y_yolo,width_yolo,height_yolo= line[0:]
#             x_yolo,y_yolo,width_yolo,height_yolo,class_id= float(x_yolo),float(y_yolo),float(width_yolo),float(height_yolo),int(class_id)
#             bbox_dict['id'] = num_bboxes
#             bbox_dict['image_id'] = file_number
#             bbox_dict['category_id'] = class_id+1
#             bbox_dict['iscrowd'] = 0 # There is an explanation before
#             h,w = abs(height_yolo*height),abs(width_yolo*width)
#             bbox_dict['area']  = h * w
#             x_coco = round(x_yolo*width -(w/2))
#             y_coco = round(y_yolo*height -(h/2))
#             if x_coco <0: #check if x_coco extends out of the image boundaries
#                 x_coco = 1
#             if y_coco <0: #check if y_coco extends out of the image boundaries
#                 y_coco = 1
#             bbox_dict['bbox'] = [x_coco,y_coco,w,h]
#             bbox_dict['segmentation'] = [[x_coco,y_coco,x_coco+w,y_coco, x_coco+w, y_coco+h, x_coco, y_coco+h]]
#             write_json_context['annotations'].append(bbox_dict)
#             num_bboxes+=1
        
#         file_number = file_number+1
#         continue
#     else:
#         continue
        
#  # Finally done, save!
# #coco_format_save_path = '/home/jyoti/Desktop/csc8800/datasets/DoorDetectDataset/test.json'
# coco_format_save_path = '/home/salem/datasets/iSAID/images/train2/test.json'
# with open(coco_format_save_path,'w') as fw:
#     json.dump(write_json_context,fw) 
    

In [ ]:
# print(sorted(os.listdir(directory_images)))
# print(img_name)
# print(img_name[:-4]+"_instance_id_RGB"+".png")


In [3]:
# Read the YOLO formatted label files (.txt) to extarct bounding box information and store in COCO format

#Read the label files (.txt) to extarct bounding box information and store in COCO format
directory_labels = os.fsencode("/home/salem/datasets/iSAID/images/train/labels")
#directory_images = os.fsencode("/home/jyoti/Desktop/csc8800/datasets/DoorDetectDataset/test")
directory_images = os.fsencode("/home/salem/datasets/iSAID/images/train/images")

file_number = 0
num_bboxes = 0
for file in sorted(os.listdir(directory_images)):
    filename = os.fsdecode(file)
#     print(filename)
    if filename.endswith(".png"):
        img_path = (os.path.join(directory_images.decode("utf-8"), filename))
        base=os.path.basename(img_path)
        file_name_without_ext = os.path.splitext(base)[0] # name of the file without the extension
        yolo_annotation_path  = os.path.join(directory_labels.decode("utf-8"), file_name_without_ext+ "." + 'txt')
        img_name = os.path.basename(img_path) # name of the file without the extension
#         print(img_name)
        img_context = {}
        height,width = cv2.imread(img_path).shape[:2]
        img_context['id'] = file_number # image id
        img_context['width'] = width
        img_context['height'] = height
        img_context['file_name'] = img_name
#         img_context['ins_file_name'] = img_name[:-4]+"_instance_id_RGB"+".png"        
#         img_context['seg_file_name'] = img_name[:-4]+"_instance_color_RGB"+".png"
        write_json_context['images'].append(img_context)
        
        with open(yolo_annotation_path,'r') as f2:
            lines2 = f2.readlines() 

        for i,line in enumerate(lines2): # for loop runs for number of annotations labelled in an image
            line = line.split(' ')
            bbox_dict = {}
            class_id, x_yolo,y_yolo,width_yolo,height_yolo= line[0:]
            x_yolo,y_yolo,width_yolo,height_yolo,class_id= float(x_yolo),float(y_yolo),float(width_yolo),float(height_yolo),int(class_id)
            
            
            h,w = abs(height_yolo*height),abs(width_yolo*width)

            x_coco = round(x_yolo*width -(w/2))
            y_coco = round(y_yolo*height -(h/2))
            if x_coco <0: #check if x_coco extends out of the image boundaries
                x_coco = 1
            if y_coco <0: #check if y_coco extends out of the image boundaries
                y_coco = 1
            
            
            bbox_dict['id'] = num_bboxes
            bbox_dict['image_id'] = file_number
            bbox_dict['segmentation'] = [[x_coco,y_coco,x_coco+w,y_coco, x_coco+w, y_coco+h, x_coco, y_coco+h]]
            bbox_dict['category_id'] = class_id+1
            a = categories[class_id]
            a = a['name']
            bbox_dict['category_name'] = a
            bbox_dict['iscrowd'] = 0 # There is an explanation before
            bbox_dict['area']  = h * w

            bbox_dict['bbox'] = [x_coco,y_coco,w,h]
            write_json_context['annotations'].append(bbox_dict)
            num_bboxes+=1
        
        file_number = file_number+1
        continue
    else:
        continue
        
 # Finally done, save!
#coco_format_save_path = '/home/jyoti/Desktop/csc8800/datasets/DoorDetectDataset/test.json'
coco_format_save_path = '/home/salem/datasets/iSAID/images/train/train.json'
with open(coco_format_save_path,'w') as fw:
    json.dump(write_json_context,fw) 
    

In [ ]:
# # Read the YOLO formatted label files (.txt) to extarct bounding box information and store in COCO format

# #Read the label files (.txt) to extarct bounding box information and store in COCO format
# directory_labels = os.fsencode("/home/salem/datasets/iSAID/images/train/labels")
# #directory_images = os.fsencode("/home/jyoti/Desktop/csc8800/datasets/DoorDetectDataset/test")
# directory_images = os.fsencode("/home/salem/datasets/iSAID/images/train/images")

# file_number = 1
# num_bboxes = 1
# for file in os.listdir(directory_images):
#     filename = os.fsdecode(file)
#     if filename.endswith(".png"):
#         img_path = (os.path.join(directory_images.decode("utf-8"), filename))
#         base=os.path.basename(img_path)
#         file_name_without_ext = os.path.splitext(base)[0] # name of the file without the extension
#         yolo_annotation_path  = os.path.join(directory_labels.decode("utf-8"), file_name_without_ext+ "." + 'txt')
#         img_name = os.path.basename(img_path) # name of the file without the extension
#         img_context = {}
#         height,width = cv2.imread(img_path).shape[:2]
#         img_context['id'] = file_number # image id
#         img_context['width'] = width
#         img_context['height'] = height
#         img_context['file_name'] = img_name
#         write_json_context['images'].append(img_context)
        
#         with open(yolo_annotation_path,'r') as f2:
#             lines2 = f2.readlines() 

#         for i,line in enumerate(lines2): # for loop runs for number of annotations labelled in an image
#             line = line.split(' ')
#             bbox_dict = {}
#             class_id, x_yolo,y_yolo,width_yolo,height_yolo= line[0:]
#             x_yolo,y_yolo,width_yolo,height_yolo,class_id= float(x_yolo),float(y_yolo),float(width_yolo),float(height_yolo),int(class_id)
            
#             h,w = abs(height_yolo*height),abs(width_yolo*width)
            
#             x_coco = round(x_yolo*width -(w/2))
#             y_coco = round(y_yolo*height -(h/2))
#             if x_coco <0: #check if x_coco extends out of the image boundaries
#                 x_coco = 1
#             if y_coco <0: #check if y_coco extends out of the image boundaries
#                 y_coco = 1
                
#             bbox_dict['id'] = num_bboxes
#             bbox_dict['image_id'] = file_number
#             bbox_dict['category_id'] = class_id+1
#             bbox_dict['iscrowd'] = 0 # There is an explanation before
#             bbox_dict['area']  = h * w

#             bbox_dict['bbox'] = [x_coco,y_coco,w,h]
#             bbox_dict['segmentation'] = [[x_coco,y_coco,x_coco+w,y_coco, x_coco+w, y_coco+h, x_coco, y_coco+h]]
#             write_json_context['annotations'].append(bbox_dict)
#             num_bboxes+=1
        
#         file_number = file_number+1
#         continue
#     else:
#         continue
        
#  # Finally done, save!
# #coco_format_save_path = '/home/jyoti/Desktop/csc8800/datasets/DoorDetectDataset/test.json'
# coco_format_save_path = '/home/salem/datasets/iSAID/images/train/test.json'
# with open(coco_format_save_path,'w') as fw:
#     json.dump(write_json_context,fw) 

In [ ]:
# def gen_anns_from_dets(dets_file, dataset_name, out_dir):
#     """Generates json annotations from detections."""

#     # Load the detections
#     with open(dets_file, 'r') as f:
#       all_boxes = pickle.load(f)['all_boxes']

#     # We assume that detections are for the given dataset
#     test_dataset = JsonDataset(dataset_name)
#     test_roidb = test_dataset.get_roidb()

#     # Generate annotations from the detections
#     anns = []

#     for cls_ind, cls in enumerate(test_dataset.classes):
#         if cls == '__background__':
#             continue
#         assert cls_ind <= len(all_boxes)

#         logger.info('Generating {} annotations ({:d}/{:d})'.format(
#             cls, cls_ind, len(all_boxes) - 1))

#         # Use original cat ids
#         cat_id = test_dataset.category_to_id_map[cls]

#         # Extract the boxes for the current category
#         cls_boxes = all_boxes[cls_ind]
#         assert len(cls_boxes) == len(test_roidb)

#         for i, entry in enumerate(test_roidb):
#             # Skip images that have no detections for the current class
#             if len(cls_boxes[i]) == 0:
#                 continue

#             boxes = cls_boxes[i].astype(np.float)
#             xs = boxes[:, 0]
#             ys = boxes[:, 1]
#             ws = boxes[:, 2] - xs + 1
#             hs = boxes[:, 3] - ys + 1
#             areas = ws * hs

#             # Construct annotations for instances detected in the ith image
#             for j in range(boxes.shape[0]):
#                 # Construct the instance annotation with box info
#                 ann_i_j = {
#                     'id': len(anns),
#                     'image_id': entry['id'],
#                     'category_id': cat_id,
#                     'segmentation': [],
#                     'area': areas[j],
#                     'bbox': [xs[j], ys[j], ws[j], hs[j]],
#                     'score': boxes[j, 4],
#                     'iscrowd': 0
#                 }
#                 anns.append(ann_i_j)

#     # Use categories from the given dataset
#     cat_ids = test_dataset.COCO.getCatIds()
#     cats = test_dataset.COCO.loadCats(cat_ids)

#     # Include the image metadata
#     img_ids = test_dataset.COCO.getImgIds()
#     imgs = test_dataset.COCO.loadImgs(img_ids)

#     # Construct a coco-style dataset object
#     gen_dataset_obj = {
#         'annotations': anns,
#         'images': imgs,
#         'categories': cats
#     }

#     # Save the generated annotations
#     out_name = 'gen_anns_{}.json'.format(test_dataset.name)
#     out_file = os.path.join(out_dir, out_name)

#     with open(out_file, 'w') as f:
#         json.dump(gen_dataset_obj, f)

In [ ]:
# gen_anns_from_dets(directory_labels, directory_images, coco_format_save_path)